[Sascha Spors](https://orcid.org/0000-0001-7225-9992),
Professorship Signal Theory and Digital Signal Processing,
[Institute of Communications Engineering (INT)](https://www.int.uni-rostock.de/),
Faculty of Computer Science and Electrical Engineering (IEF),
[University of Rostock, Germany](https://www.uni-rostock.de/en/)

# Tutorial Signals and Systems (Signal- und Systemtheorie)

Summer Semester 2023 (Bachelor Course #24015)

- lecture: https://github.com/spatialaudio/signals-and-systems-lecture
- tutorial: https://github.com/spatialaudio/signals-and-systems-exercises

Feel free to contact lecturer [frank.schultz@uni-rostock.de](https://orcid.org/0000-0002-3010-0294)

## Übung / Exercise 5 Magnitude Bode Plot Approximation for LTI Systems

### References:

* Norbert Fliege (1991): "*Systemtheorie*", Teubner, Stuttgart (GER), cf. chapter 4.3.5

* Alan V. Oppenheim, Alan S. Willsky with S. Hamid Nawab (1997): "*Signals & Systems*", Prentice Hall, Upper Saddle River NJ (USA), 2nd ed., cf. chapter 6

* Bernd Girod, Rudolf Rabenstein, Alexander Stenger (2001): "*Signals and Systems*", Wiley, Chichester (UK), cf. chapter 10

* Bernd Girod, Rudolf Rabenstein, Alexander Stenger (2005/2007): "*Einführung in die Systemtheorie*", Teubner, Wiesbaden (GER), 3rd/4th ed., cf. chapter 10

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
decades = 2  # plot number of decades left and right from cut frequency

print('log10 axis spacing:')
print(np.log10(np.arange(1, 11, 1)))
print('log2 axis spacing:')
print(np.log2(np.arange(1, 2.1, 0.1)))

## 2nd Order Lowpass, PT2

The 2nd order lopwass from `laplace_transform/solving_2nd_order_ode.tex`
\begin{align}
H_\mathrm{Low}(s) = \frac{1}{\frac{16}{25}s^2+\frac{24}{25}s +1} = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}
\end{align}
is to be visualized as approximated and exact magnitude Bode plot.
See `laplace_system_analysis/frequency_response_2nd_order_ode.tex` for detailed calculus and discussion.

\begin{align}
H(s) = H_0 \frac{1}{(s-s_{\infty,1}) (s-s_{\infty,2})}
\end{align}

The poles 

$s_{\infty,1,2} = -\frac{3}{4} \pm \mathrm{j}$

constitute a complex conjugate pair, thus

$|s_{\infty,1}| = |s_{\infty,2}| = \frac{5}{4}$, $|s_{\infty,1}|^2 = |s_{\infty,2}|^2 = \frac{25}{16}$

$H_0 = 25/16$.

\begin{align}
\tilde{H_0} = \frac{H_0}{|s_{\infty,1}|^2} = \frac{\frac{25}{16}}{\frac{25}{16}} = 1 \rightarrow 0 \mathrm{dB}
\end{align}


In [ ]:
w = np.arange(1e-2, 1e+2, 0.01)
s = 1j*w

# define LTI system by zeros and poles:
soo1 = (-3/4+1j)
soo2 = (-3/4-1j)
H0 = 25/16
# get transfer function Python/Matlab-like handling:
sz = 0  # we need this zero and the pole at the origin for signal.lti()
sp = 0, soo1, soo2
sys = signal.lti(sz, sp, H0)
w, Hmag, Hphase = sys.bode(w)

# Bode approximation:
# gain -> straight line at 20 dB, blue:
H0tilde = H0 / np.abs(soo1)**2
H0t = np.zeros_like(w) + 20*np.log10(np.abs(H0tilde))
# conj complex pole ->
Hsoo1 = -2*20*np.log10(np.abs(s/soo1))  # line with -40dB/decade for w>5/4, red
Hsoo1[np.abs(s) <= np.abs(soo1)] = 0  # 0 dB for w<5/4, red
# add dB
Happrox = H0t + Hsoo1  # add magnitude frequency responses in dB

plt.figure(figsize=(7, 5))
plt.semilogx(w, H0t, color='C0', lw=5,
             label=r'$|\tilde{H_0}|=1\rightarrow 0$ dB')
plt.semilogx(w, Hsoo1, color='C3', lw=4,
             label=r'conj complex pole $|s_{\infty,1,2}|=\frac{5}{4}$')
plt.semilogx(w, Happrox, color='k', lw=2, label='Bode plot approximation')
plt.semilogx(w, Hmag, lw=2, color='C8', label='Bode plot exact')

plt.xticks((1e-1, 2e-1, 5e-1, 1e0, 5/4, 2, 2.5, 5, 10, 12.5, 20, 50, 100),
           ['0.1', '0.2', '0.5', '1', r'$\frac{5}{4}$', '2', r'$\frac{5}{2}$', '5', '10', '12.5', '20', '50', '100'])
plt.yticks((-40, -36, -30, -24, -20, -12, -10, -6, -3, 0, 3, 6, 10))
plt.xlim(0.1, 13)
plt.ylim(-40, 3)
plt.xlabel(r'$\omega$ / (rad/s)')
plt.ylabel('level: 20 lg|H| / dB')
plt.title(
    r'magnitude frequency response of $H(s) = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}$')
plt.legend()
plt.grid(True, which='both')
plt.savefig('bodeplot_example_approximations_level_44EB4169E9.pdf')

In [ ]:
w = np.arange(1e-2, 1e+2, 0.01)
s = 1j*w

sz = 0
sp = 0, -3/4-1j, -3/4+1j
H0 = 25/16
sys = signal.lti(sz, sp, H0)
w, mag, phase = sys.bode(np.arange(1e-2, 1e2, 1e-2))

plt.figure(figsize=(7, 5))
plt.semilogx(w, phase, 'C8', linewidth=3)
plt.xticks((1e-1, 2e-1, 5e-1, 1e0, 5/4, 2, 2.5, 5, 10, 12.5, 20, 50, 100),
           ['0.1', '0.2', '0.5', '1', r'$\frac{5}{4}$', '2', r'$\frac{5}{2}$', '5', '10', '12.5', '20', '50', '100'])
plt.yticks(np.arange(-180, 225, 45))
plt.xlim(0.1, 13)
plt.ylim(-180, 0)
plt.xlabel('$\omega$ / (rad/s)')
plt.ylabel(r'phase: $\angle(H)$ / deg')
plt.title(
    r'phase frequency response of $H(s) = [\frac{16}{25}s^2+\frac{24}{25}s +1]^{-1}$')
plt.grid(True, which='both')
plt.savefig('bodeplot_example_approximations_phase_44EB4169E9.pdf')

## 2nd Order Lowpass, PT2 -> Varying Pole Quality

When using cut frequency $\omega_0$, damping factor $0 < D \leq 1$ or pole quality $Q>0.5$ we can write the Laplace transfer function 
of the 2nd order lowpass filter as
\begin{align}
H(s) = \frac{1}{\frac{1}{\omega_0^2} s^2 + \frac{2 D}{\omega_0} s + 1}=
\frac{1}{\frac{1}{\omega_0^2} s^2 + \frac{1}{\omega_0 Q_\infty} s + 1}.
\end{align}
With the example below you can play around with w0 (i.e. $\omega_0$) and D.

Note that this semi-logarithmic plot is over the **frequency** $f$ in Hz, rather than over **angular frequency** $\omega$ in rad/s.

In [ ]:
# user input:
w0 = 2*np.pi*1  # cut frequency in rad/s
# 0 < D <= 1:
# D = 1/100  # near to the jw axis, more resonsant
#D = 1/10
D = 3/5
#D = 1/np.sqrt(2)
# D = 1  # far from jw axis, less resonsant
###

Q = 1/(2*D)
soo1 = - D*w0 + 1j*w0*np.sqrt(1-D**2)
soo2 = - D*w0 - 1j*w0*np.sqrt(1-D**2)
print('D = %4.3f, Q = %4.3f' % (D, Q))
print('poles s001,2 = %5.4f +- j %5.4f' % (np.real(soo1), np.imag(soo1)))

w = np.arange(w0/10**decades, w0*10**decades, w0/100)
s = 1j*w

# exact:
H = 1/(s**2/w0**2 + s/w0/Q + 1)
Hlevel = 20*np.log10(np.abs(H))

# approximation:
wconst = w[np.abs(s) < w0]
Hconst = 0*wconst
wslope = np.array([w0, w[-1]])
Hslope = (0, -40*decades)

# plot
plt.figure(figsize=(7, 5))
plt.semilogx(wslope/2/np.pi, Hslope, color='k', lw=3)
plt.semilogx(wconst/2/np.pi, Hconst, color='k',
             label='Bode plot approximation', lw=3)
plt.semilogx(w/2/np.pi, Hlevel, color='C8',
             label=r'Bode plot exact for $Q_{\infty,1}=$%5.4f' % Q, lw=2)
plt.xlabel('f / Hz')
plt.ylabel('level: 20 lg|H| / dB')
plt.yticks((-60, -40, -20, -12, 0, 6))
plt.title(r'magnitude frequency response of 2nd order lowpass')
plt.legend()
plt.grid(True, which='both')
plt.xlim(w[0]/2/np.pi, w[-1]/2/np.pi)

## 2nd Order Bandpass

We assume the causal LTI system with

* zero in origin $s_0=0$
* pole at $s_{\infty,1}=-0.1$
* pole at $s_{\infty,2}=-10$
* gain $H_0=10$

and want to create the magnitude Bode plot approximation.

\begin{align}
H(s) = H_0\frac{s-s_{0,1}}{(s-s_{\infty,1})(s-s_{\infty,2})} = 10\frac{(s-0)}{(s-(-0.1))(s-(-10))}
=\frac{100 s}{10 s^2 + 101 s + 10}
\end{align}

\begin{align}
20\mathrm{lg}|H(s)| = 20\mathrm{lg}|\tilde{H_0}| + 20\mathrm{lg}|s| - 20\mathrm{lg}|\frac{s}{|s_{\infty,1}|}-1| 
- 20\mathrm{lg}|\frac{s}{|s_{\infty,2}|}-1| 
\end{align}

\begin{align}
\tilde{H_0} = \frac{H_0}{|s_{\infty,1}| \cdot |s_{\infty,2}|} = 10 \rightarrow 20\mathrm{lg}|\tilde{H_0}| = 20 \mathrm{dB}
\end{align}

In [ ]:
w = np.arange(1e-2, 1e+2, 0.01)
s = 1j*w
# define LTI system by zeros and poles:
s01 = 0
soo1 = -1/10
soo2 = -10
H0 = 10

# get transfer function Python/Matlab-like handling:
sz = s01
sp = soo1, soo2
sys = signal.lti(sz, sp, H0)
w, Hexact, Hphase = sys.bode(w)
# tf = sys.to_tf()  # get b=num,a=den coefficients
# H0 = tf.num[0] / tf.den[0]  # get H0 from the polynomial version

# Bode approximation:

# gain -> straight line at 20 dB, blue
H0tilde = H0 / np.abs(soo1) / np.abs(soo2)
H0t = np.zeros_like(w) + 20*np.log10(np.abs(H0tilde))

# zero in origin -> line with +20dB/decade through 1 rad/s, orange
Hs01 = +20*np.log10(np.abs(s))

# single pole ->
Hsoo1 = -20*np.log10(np.abs(s/soo1))  # line with -20dB/decade for w>0.1, green
Hsoo1[np.abs(s) <= np.abs(soo1)] = 0  # 0 dB for w<0.1, green

# single pole ->
Hsoo2 = -20*np.log10(np.abs(s/soo2))  # line with -20dB/decade for w>10, red
Hsoo2[np.abs(s) <= np.abs(soo2)] = 0  # 0 dB for w<10, red

Happrox = H0t + Hs01 + Hsoo1 + Hsoo2  # add magnitude frequency responses in dB

Hexact2 = 20*np.log10(H0tilde) + 20*np.log10(np.abs(s)) - 20*np.log10(
    np.abs(s/np.abs(soo1)-1)) - 20*np.log10(np.abs(s/np.abs(soo2)-1))

plt.figure(figsize=(7, 5))
plt.semilogx(w, H0t, color='C0', lw=2,
             label=r'$|\tilde{H_0}|=10\rightarrow 20$ dB')
plt.semilogx(w, Hs01, color='C1', lw=2, label='one zero in origin')
plt.semilogx(w, Hsoo1, color='C2', lw=5,
             label=r'real, single pole at $|s_{\infty,1}|=\frac{1}{10}$')
plt.semilogx(w, Hsoo2, color='C3', lw=3,
             label=r'real, single pole at $|s_{\infty,2}|=10$')
plt.semilogx(w, Happrox, color='k', label='Bode plot approximation')
plt.semilogx(w, Hexact2, lw=1, color='C8')
plt.semilogx(w, Hexact, lw=1, color='C8',
             label='Bode plot exact')

plt.xlim(1e-2, 1e2)
plt.yticks((-60, -40, -20, -10, -3, -6, 0, 6, 20, 40))
plt.ylim(-60, 40)
plt.xlabel(r'$\omega$ / (rad/s)')
plt.ylabel('20 lg|H| / dB')
plt.title(r'magnitude frequency response of $H(s) = 100s\,/\,(10s^2+101s+10)$')
plt.legend()
plt.grid(True, which='both')
plt.savefig('bodeplot_example_approximations_590A7AFD51.pdf')

## Highpass 2nd Order with Slight Resonance

We assume the causal LTI system 
\begin{align}H(s) = \frac{s^2+10 s}{s^2+\sqrt{2} s +1}\end{align}
and want to create the magnitude Bode plot approximation.

\begin{align}
&s_{0,1} = 0\rightarrow |s_{0,1}| = 0\\
&s_{0,2} = -10 \rightarrow |s_{0,2}| = 10\\
&s_{\infty,1,2} = -\frac{\sqrt{2}}{2} \pm \sqrt{\frac{2}{4}-1} = -\frac{1}{\sqrt{2}} \pm \text{j} \frac{1} {\sqrt{2}} \rightarrow |s_{\infty,1,2}|^2 = 1\\
&|H_0| = 1\\
&|\tilde{H}_0| = 1 \cdot \frac{|s_{0,2}|}{|s_{\infty,1,2}|^2} = 10 \rightarrow 20\text{lg}|\tilde{H}_0|=20 \text{dB}
\end{align}

In [ ]:
w = np.arange(1e-2, 1e+2, 0.01)
s = 1j*w
# define LTI system by zeros and poles:
s01 = 0
s02 = 10
soo1 = (-1+1j)/np.sqrt(2)
soo2 = (-1-1j)/np.sqrt(2)
H0 = 1
# get transfer function Python/Matlab-like handling:
sz = s01, s02
sp = soo1, soo2
sys = signal.lti(sz, sp, H0)
w, Hmag, Hphase = sys.bode(w)
# tf = sys.to_tf()  # get b=num,a=den coefficients
# H0 = tf.num[0] / tf.den[0]  # get H0 from the polynomial version

# Bode approximation:

# gain -> straight line at 20 dB, blue
H0tilde = H0 * np.abs(s02) / np.abs(soo1)**2
H0t = np.zeros_like(w) + 20*np.log10(np.abs(H0tilde))

# single zero in origin -> line with +20dB/decade through 1 rad/s, orange
Hs01 = +20*np.log10(np.abs(s))

# single zero ->
Hs02 = +20*np.log10(np.abs(s/s02))  # line with +20dB/decade for w>10, green
Hs02[np.abs(s) <= np.abs(s02)] = 0  # 0 dB for w<10, green

# conj complex pole ->
Hsoo1 = -2*20*np.log10(np.abs(s/soo1))  # line with -40dB/decade for w>1, red
Hsoo1[np.abs(s) <= np.abs(soo1)] = 0  # 0 dB for w<1, red

Happrox = H0t + Hs01 + Hs02 + Hsoo1  # add magnitude frequency responses in dB

plt.figure(figsize=(7, 5))
plt.semilogx(w, H0t, color='C0', lw=2,
             label=r'$|\tilde{H_0}|=10\rightarrow 20$ dB')
plt.semilogx(w, Hs01, color='C1', lw=2, label=r'zero in origin $s_{0,1}$')
plt.semilogx(w, Hs02, color='C2', lw=3, label=r'zero $|s_{0,2}|=10$')
plt.semilogx(w, Hsoo1, color='C3', lw=2,
             label='conj complex pole $|s_{oo,1,2}|=1$')
plt.semilogx(w, Happrox, color='k', lw=2, label='Bode Plot Approximation')
plt.semilogx(w, Hmag, lw=1, color='C8', label='Bode Plot Exact')
plt.xlim(1e-2, 1e2)
plt.ylim(-60, 40)
plt.yticks((-60, -40, -20, -12, -6, 0, 6, 12, 20, 40))
plt.xlabel(r'$\omega$ / (rad/s)')
plt.ylabel('20 lg|H| / dB')
plt.title(
    r'magnitude frequency response of $H(s) = (s^2+10 s)\,/\,(s^2 + \sqrt{2} s +1)$')
plt.legend()
plt.grid(True, which='both')
plt.savefig('bodeplot_example_approximations_1EC316D8B2.pdf')

## Copyright

This tutorial is provided as Open Educational Resource (OER), to be found at
https://github.com/spatialaudio/signals-and-systems-exercises
accompanying the OER lecture
https://github.com/spatialaudio/signals-and-systems-lecture.
Both are licensed under a) the Creative Commons Attribution 4.0 International
License for text and graphics and b) the MIT License for source code.
Please attribute material from the tutorial as *Frank Schultz,
Continuous- and Discrete-Time Signals and Systems - A Tutorial Featuring
Computational Examples, University of Rostock* with
``github URL, commit number and/or version tag, year, (file name and/or content)``.